## Convert *existing* paradigm (PsychToolbox input) into FreeSurfer-friendly .par files

In [ ]:
import os
import pandas as pd
import numpy as np

behav_dir='/autofs/space/lilli_001/users/DARPA-newARC/behavior/params'
par_path='/autofs/space/lilli_001/users/DARPA-newARC/behavior/params/optseq'

df = pd.read_csv(os.path.join(par_path,'mri_newarc.csv'),index_col=False)
df=pd.DataFrame(df)

ConflictType=[]
ITI=[]
Reward=[]
Risk=[]
Trial=[]
Conflict=[]
## Replace Slope with log reg results
Slope=np.zeros(2*len(df))


for i in df.ConflictType:
    ConflictType.append('NULL     ')
    ConflictType.append(i)
    
for i in df.ITI:
    ITI.append(i) 
    ITI.append(5.25)

Time=[0]
s=0

for i in ITI:
    s=s+i
    Time.append(s)

Time=Time[:-1]

for ct in zip(ConflictType):
    if 'lowconflict' in ct:
        Conflict.append('1')
    elif 'medconflict' in ct:
        Conflict.append('2')
    elif 'highconflict' in ct:
        Conflict.append('3')
    elif 'NULL     ' in ct:
        Conflict.append('0')
    else: 
        print ct

df2_data={'Time':Time,'Conflict':Conflict,'ITI':ITI,'Slope':Slope,'ConflictType':ConflictType}
df2=pd.DataFrame(df2_data)

# print len(df2_data['Time'])
# print len(df2_data['Conflict'])
# print len(df2_data['ITI'])
# print len(df2_data['Slope'])
# print len(df2_data['ConflictType'])

trial_1 = pd.DataFrame(df2.iloc[0:162])
trial_2 = pd.DataFrame(df2.iloc[162:324])
trial_3 = pd.DataFrame(df2.iloc[324::])

trial_1.to_csv(os.path.join(par_path,'3cond_mod-001.csv'),index=False)
trial_2.to_csv(os.path.join(par_path,'3cond_mod-002.csv'),index=False)
trial_3.to_csv(os.path.join(par_path,'3cond_mod-003.csv'),index=False)
df2.to_csv(os.path.join(par_path,'3cond_mod-AllRuns.csv'),index=False)

## Configure files to be FreeSurfer Compatible

In [ ]:
import os
import pandas as pd
import numpy as np

bdir='/autofs/space/lilli_001/users/DARPA-newARC/behavior/behav_runs'
indiv_par_path='/autofs/space/lilli_001/users/DARPA-newARC/behavior/indiv_par_files'
par_path='/autofs/space/lilli_001/users/DARPA-newARC/behavior/params/optseq'

# subjects = ['newARC_001','newARC_004','newARC_005','newARC_008',\
#             'newARC_009','newARC_010','newARC_011','newARC_014',\
#             'newARC_015','newARC_016','newARC_017','newARC_018',\
#             'newARC_020']

subjects = ['newARC_001','newARC_004','newARC_005','newARC_008',\
            'newARC_009','newARC_010','newARC_011','newARC_014',\
            'newARC_015','newARC_016','newARC_017','newARC_018']

for subject in subjects:
    csv=os.path.join(bdir,'%s_run_1.csv' % subject)
    onsets=[]
    df=pd.DataFrame.from_csv(csv)
    for fix_on,rew_on in zip(df['FixOnset'],df['RewardOnset']):
        onsets.append(fix_on)
        onsets.append(rew_on)
    run1=os.path.join(par_path,'3cond_mod-001.csv')
    df1=pd.read_csv(run1)
    df1=pd.DataFrame(df1,columns=['Time','Conflict','ITI','Slope','ConflictType'])
    df1['Time'] = onsets
    display(df1)
        
    trial_1.to_csv(os.path.join(indiv_par_path,'%s_run1.par'% subject),sep='\t',index=False,header=None)

## Concatenate par files